In [8]:
import os
import sys
import torch

sys.path.append(os.getcwd())
from lossless.component.coolchic import CoolChicEncoderParameter
from lossless.component.coolchic import CoolChicEncoder
from lossless.util.config import (
    args,
)
from lossless.util.parsecli import (
    change_n_out_synth,
    get_coolchic_param_from_args,
)
from lossless.nnquant.quantizemodel import quantize_model


encoder_param = CoolChicEncoderParameter(
    **get_coolchic_param_from_args(args, "residue")
)
encoder_param.set_image_size((512, 768))
encoder_param.layers_synthesis = change_n_out_synth(
    encoder_param.layers_synthesis, args["output_dim_size"]
)

coolchic = CoolChicEncoder(param=encoder_param)
coolchic.to_device("cuda:0")

In [9]:
# print(coolchic.get_flops())
# print(coolchic.flops_str())
with torch.no_grad():
    print(coolchic.get_total_mac_per_pixel())
    print(coolchic.str_complexity())

3891.5703125
| module                            | #parameters or shape   | #flops     |
|:----------------------------------|:-----------------------|:-----------|
| model                             | 0.528M                 | 1.53G      |
|  latent_grids                     |  0.524M                |            |
|   latent_grids.0                  |   (1, 1, 512, 768)     |            |
|   latent_grids.1                  |   (1, 1, 256, 384)     |            |
|   latent_grids.2                  |   (1, 1, 128, 192)     |            |
|   latent_grids.3                  |   (1, 1, 64, 96)       |            |
|   latent_grids.4                  |   (1, 1, 32, 48)       |            |
|   latent_grids.5                  |   (1, 1, 16, 24)       |            |
|   latent_grids.6                  |   (1, 1, 8, 12)        |            |
|  synthesis.layers                 |  2.301K                |  0.875G    |
|   synthesis.layers.0              |   0.384K               |   0.132G   |